In [3]:
from googleapiclient.discovery import build
from datetime import datetime
import os
import json
# Datos de tu proyecto de Google Cloud
API_KEY = os.getenv('YOUTUBE_API_KEY')
CHANNEL_ID = 'UC7mJ2EDXFomeDIRFu5FtEbA'

# Inicializa el cliente de la API
youtube = build('youtube', 'v3', developerKey=API_KEY)

# Establecer la fecha desde la cual buscar los videos
published_after = datetime(2024, 9, 7).isoformat("T") + "Z"

# Realiza la consulta a la API para obtener los videos
request = youtube.search().list(
    part='snippet',
    channelId=CHANNEL_ID,
    publishedAfter=published_after,
    order='date',  # Ordenar por fecha
    type='video',  # Asegurarse de que solo busca videos
    maxResults=1  # Puedes ajustar esto según tus necesidades
)

response = request.execute()

print(json.dumps(response['items']),indent = 4)


TypeError: 'indent' is an invalid keyword argument for print()

In [5]:

print(json.dumps(response,indent = 4))


{
    "kind": "youtube#searchListResponse",
    "etag": "tCj1FOheDEdZdaRqmj7KFqBo1i4",
    "nextPageToken": "CAEQAA",
    "regionCode": "AR",
    "pageInfo": {
        "totalResults": 174,
        "resultsPerPage": 1
    },
    "items": [
        {
            "kind": "youtube#searchResult",
            "etag": "tj23L4f2lUL4MojvcHwD2J1RUGs",
            "id": {
                "kind": "youtube#video",
                "videoId": "YaaskpMWLYY"
            },
            "snippet": {
                "publishedAt": "2024-09-21T12:00:36Z",
                "channelId": "UC7mJ2EDXFomeDIRFu5FtEbA",
                "title": "NATI Y BETU CANCELADOS POR LOS GUSTOS DE HELADO",
                "description": "betular #natijota #eialmoldavsky #homeropettinato #olgaenvivo #olga #streaming #stream #shortclip #shorts.",
                "thumbnails": {
                    "default": {
                        "url": "https://i.ytimg.com/vi/YaaskpMWLYY/default.jpg",
                        "width": 120,
 

In [ ]:

published_after =  datetime(2024, 9, 15).isoformat("T") + "Z"
request = youtube.search().list(
        part='snippet',
        channelId='UC7mJ2EDXFomeDIRFu5FtEbA',
        publishedAfter=published_after,
        order='date',
        type='video',
    )

response = request.execute()
response['items']

In [3]:
from utils import initialize_youtube_api, get_videos_from_channel, transform_video_data, group_videos_by_date, CHANNEL_IDS
from datetime import datetime
import pandas as pd
import sys

"""
    Tiene tres etapas:
    iterar por cada channel y traerme la información sobre qué videos se publicaron
    iterar sobre los videos y traernos las statistics y las metemos en un dataframe
    subimos el dataframe a redshift
    
"""
# Inicializamos la API
youtube = initialize_youtube_api()

# Establezco la fecha de inicio (últimos N días)
published_after = datetime(2024, 9, 15).isoformat("T") + "Z"

all_videos = []

# Para cada channel de youtube itero y me traigo los videos
for channel_id in CHANNEL_IDS:
    videos = get_videos_from_channel(youtube, channel_id, published_after)
    transformed_videos = transform_video_data(videos)
    all_videos.extend(transformed_videos)

# Agrupo por fecha
grouped_videos = group_videos_by_date(all_videos)

# Convertir los datos agrupados en una lista de diccionarios para DataFrame
video_data = []
for date, videos in grouped_videos.items():
    for video in videos:
        video_data.append({
            'date': date,
            'title': video['title'],
            'published_at': video['published_at']
        })

# Guardamos los datos en un DataFrame
df = pd.DataFrame(video_data)

# Guardar en un archivo CSV
df.to_csv('youtube_videos.csv', index=False)

# Mostrar los primeros registros del DataFrame para verificar
print(df.head())

         date                                              title  \
0  2024-09-18                       NOLY Y SU ÚLTIMO DÍA DE VIDA   
1  2024-09-18  🔴 MALDICION, VA A SER UN DIA HERMOSO con Mario...   
2  2024-09-18  VINO LEILA A BANCAR | #QUÉROMPIMOS Completo - ...   
3  2024-09-18  CUMPLE ALFRE, DEBATE DE LAS ESTRELLAS DEPORTIV...   
4  2024-09-18  #EDICIONESPECIAL | MICAELA&#39;S, MEJORES AMIG...   

           published_at  
0  2024-09-18T10:00:14Z  
1  2024-09-18T10:10:28Z  
2  2024-09-18T02:02:09Z  
3  2024-09-18T00:45:01Z  
4  2024-09-18T03:17:34Z  


In [1]:

import os

# Inicializamos la API
API_KEY = os.getenv('YOUTUBE_API_KEY')
youtube = build('youtube', 'v3', developerKey=API_KEY)

# Realizamos la búsqueda del canal por handle (nombre del canal)
request = youtube.search().list(
    part='snippet',
    q='@VorterixOficial',
    type='channel',
    maxResults=1  # Solo necesitamos el canal
)

response = request.execute()

# Extraemos el ID del canal de la respuesta
for item in response['items']:
    channel_id = item['snippet']['channelId']
    print(f"ID del canal: {channel_id}")

NameError: name 'build' is not defined

In [19]:
import time
from googleapiclient.errors import HttpError
import isodate

def convert_duration_to_seconds(duration_iso):
    # Convierte la duración ISO 8601 a un objeto timedelta
    duration = isodate.parse_duration(duration_iso)
    return int(duration.total_seconds())

def get_channel_info(youtube, channel_id):
    try:
        request = youtube.channels().list(
            part='snippet,statistics',
            id=channel_id
        )
        response = request.execute()

        # Extraemos la información del canal
        channel_info = response['items'][0]
        channel_data = {
            'channel_name': channel_info['snippet']['title'],
            'channel_id': channel_info['id'],
            'subscriber_count': channel_info['statistics'].get('subscriberCount', 0)
        }

        return channel_data

    except HttpError as e:
        print(f"Error al obtener información del canal {channel_id}: {e}")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return None
    
    
def get_videos_from_channel(youtube, channel_id, published_after, max_requests=10, sleep_time=1):
    videos = []
    request_count = 0
    
    try:
        request = youtube.search().list(
            part='snippet',
            channelId=channel_id,
            publishedAfter=published_after,
            order='date',
            type='video',
            maxResults=10  # Puedes ajustar según tus necesidades
        )
        response = request.execute()

        # Procesamos la respuesta
        for item in response['items']:
            video_data = {
                'title': item['snippet']['title'],
                'published_at': item['snippet']['publishedAt'],
                'video_id': item['id']['videoId'],
                'video_type': item['snippet'].get('liveBroadcastContent', 'none')  # live, upcoming o none
            }
            videos.append(video_data)
        
        request_count += 1
        
        # Si hemos alcanzado el límite de solicitudes, hacemos un sleep
        if request_count % max_requests == 0:
            print(f"Realizadas {request_count} solicitudes. Esperando {sleep_time} segundos para continuar...")
            time.sleep(sleep_time)

    except HttpError as e:
        print(f"Error al obtener videos del canal {channel_id}: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

    return videos

def get_video_statistics(youtube, video_ids, max_requests=10, sleep_time=1):
    video_stats = []
    request_count = 0
    
    try:
        stats_request = youtube.videos().list(
            part='statistics,contentDetails',
            id=','.join(video_ids)  # Pasamos los IDs de los videos como una lista separada por comas
        )
        stats_response = stats_request.execute()

        # Procesamos las estadísticas
        for stats in stats_response['items']:
            # Convertimos la duración de ISO 8601 a segundos
            duration_iso = stats['contentDetails']['duration']
            duration_seconds = convert_duration_to_seconds(duration_iso)
            
            # Verificamos si el video es un YouTube Short (menos de 60 segundos)
            is_short = duration_seconds < 60
            
            stats_data = {
                'video_id': stats['id'],
                'view_count': stats['statistics'].get('viewCount', 0),
                'like_count': stats['statistics'].get('likeCount', 0),
                'comment_count': stats['statistics'].get('commentCount', 0),
                'duration': duration_seconds,
                'is_short': is_short
            }
            video_stats.append(stats_data)
        
        request_count += 1

        # Si hemos alcanzado el límite de solicitudes, hacemos un sleep
        if request_count % max_requests == 0:
            print(f"Realizadas {request_count} solicitudes. Esperando {sleep_time} segundos para continuar...")
            time.sleep(sleep_time)

    except HttpError as e:
        print(f"Error al obtener estadísticas de videos: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

    return video_stats

In [20]:

# Obtenemos la información del canal
channel_info = get_channel_info(youtube, channel_id)


# Obtener la fecha de consulta actual
consulta_fecha = datetime.now().strftime('%Y-%m-%d')

# Crear un DataFrame con la información de suscriptores del canal
df_subscribers = pd.DataFrame([{
    'channel_name': channel_info['channel_name'],
    'channel_id': channel_info['channel_id'],
    'consulta_fecha': consulta_fecha,
    'subscriber_count': channel_info['subscriber_count']
}])

# Verificamos que se haya obtenido la información del canal
if channel_info:
    # Creamos una lista vacía para almacenar los datos combinados
    combined_data = []

    # Iteramos sobre los videos y sus estadísticas
    for video, stats in zip(videos, video_stats):
        # Combinamos la información de los videos, las estadísticas y el canal en un solo diccionario
        combined_data.append({
            'channel_name': channel_info['channel_name'],
            'channel_id': channel_info['channel_id'],
            'title': video['title'],
            'published_at': video['published_at'],
            'video_id': video['video_id'],
            'video_type': video['video_type'],
            'view_count': stats['view_count'],
            'like_count': stats['like_count'],
            'comment_count': stats['comment_count'],
            'duration_seconds': stats['duration'],
            'is_short': stats['is_short']
        })

    # Convertimos la lista de diccionarios en un DataFrame de pandas
    df_videos = pd.DataFrame(combined_data)

In [23]:
df_videos['likes_per_view'] = df_videos['like_count'] / df_videos['view_count']
df_videos['comments_per_view'] = df_videos['comment_count'] / df_videos['view_count']

,channel_name,channel_id,title,published_at,video_id,video_type,view_count,like_count,comment_count,duration_seconds,is_short
0,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,MIRANDA Y UN TEMAZO DE CRISTIAN CASTRO,2024-09-21T14:00:35Z,XjEX99nbf6Q,none,35,15,0,36,True
1,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,NATI Y BETU CANCELADOS POR LOS GUSTOS DE HELADO,2024-09-21T12:00:36Z,YaaskpMWLYY,none,573,127,1,60,False
2,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,GREGO DE CHETO A HUMILDE POR UNA TORTA FRITA,2024-09-21T10:00:15Z,HzU45a7Pxjc,none,1135,83,0,57,True
3,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,OLIVIA FIRPO CUMPLIÓ SU SUEÑO EN EL TEATRO COLÓN,2024-09-21T00:19:15Z,F7ZejuxyrDw,none,4344,273,2,55,True
4,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,CARO Y SOFI GONET DEL ODIO AL AMOR POR LA MODA,2024-09-20T20:00:17Z,ma15lzAoKx4,none,1498,103,0,60,False
5,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,AGUANTE OEISIS | Soñé que Volaba | COMPLETO 20/9,2024-09-20T18:07:37Z,OZeLwvFPfGk,none,56980,1649,182,8283,False
6,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,RECHI Y EL MEJOR CHISTE DE LA HISTORIA,2024-09-20T18:00:26Z,i3YkcpDXcP4,none,11893,769,6,60,False
7,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,ESPECIAL SUPERCLÁSICO: BOCA - RIVER en OLGA co...,2024-09-20T17:52:47Z,XXqVT9gaw_E,upcoming,0,8,0,0,True
8,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,GIME SE PELEÓ CON TODOS,2024-09-20T16:00:45Z,gFhK3npHMRo,none,2244,202,0,59,True
9,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,INTELIGENCIA ARTIFICIAL y la PREVIA del SUPERC...,2024-09-20T15:24:24Z,5xcXpVf3IPU,none,86932,2380,149,7261,False


In [55]:
from utils import initialize_youtube_api, get_videos_from_channel, get_video_statistics, get_channel_info, group_videos_by_date, CHANNEL_IDS
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import pandas as pd

"""
    Tiene tres etapas:
    1. Iterar por cada channel y traer la información sobre qué videos se publicaron
    2. Iterar sobre los videos y traernos las estadísticas y las metemos en un DataFrame
    3. Subir los DataFrames (videos y suscriptores) a Redshift o guardarlos en CSV
    
    WARNING: Este código es para correr de manera diaria.
    Una vulnerabilidad que tiene es que la tabla con cantidad de suscriptores si se corre desde una fecha en particular, no va a obtener los subs desde esa fecha, si no de manera diaria
    
"""

# Inicializamos la API
youtube = initialize_youtube_api()

# Establezco la fecha de inicio (últimos N días)
published_after = (datetime.now() - timedelta(days=7)).isoformat("T") + "Z"

all_videos = []
all_video_stats = []
df_subscribers_list = []



# Se itera para cada channel de YouTube y sacamos estadísticas: cantindad de subs a la fecha de consulta, sus videos y para cada video métricas de views, comentarios y likes
for channel_id in CHANNEL_IDS:
    # Obtenemos la información del canal (nombre, id, suscriptores)
    channel_info = get_channel_info(youtube, channel_id)
    
    # Obtener la fecha de consulta actual
    consulta_fecha = datetime.now().strftime('%Y-%m-%d')
    
    # Crear un df con la info de suscriptores del canal
    df_subscribers_list.append({
        'channel_name': channel_info['channel_name'],
        'channel_id': channel_info['channel_id'],
        'consulta_fecha': consulta_fecha,
        'subscriber_count': channel_info['subscriber_count']
    })

    videos = get_videos_from_channel(youtube, channel_id, published_after)
    
    # Filtramos videos que contengan alguno de los campos que necesitamos para obtener el video ID. 
    # Hacemos esto por si cambia el nombre de la key con la que se guarda el video_id. 
    video_ids = []
    for video in videos:
        if 'video_id' in video:
            video_ids.append(video['video_id'])
        elif 'id' in video and isinstance(video['id'], dict) and 'videoId' in video['id']:
            video_ids.append(video['id']['videoId'])
        else:
            # Si no se encuentra el video_id, imprimimos el video para depuración
            print(f"Video sin 'video_id' o 'id' válido detectado: {video}")
    
    # Las stats de los videos
    video_stats = get_video_statistics(youtube, video_ids)
    
    # Agregamos los videos y las estadísticas a una lista combinada
    for video, stats in zip(videos, video_stats):
        video_id = video.get('video_id') or (video['id']['videoId'] if isinstance(video['id'], dict) and 'videoId' in video['id'] else 'N/A')
        
        all_videos.append({
            'channel_name': channel_info['channel_name'],
            'channel_id': channel_info['channel_id'],
            'title': video['title'],
            'published_at': video['published_at'],
            'video_id': video['video_id'],
            'video_type': video['video_type'],
            'view_count': stats['view_count'],
            'like_count': stats['like_count'],
            'comment_count': stats['comment_count'],
            'duration_seconds': stats['duration'],
            'is_short': stats['is_short']
        })

# Pasamos a un df los videos
df_videos = pd.DataFrame(all_videos)

# Hago una transformación de los datos 
df_videos['like_count'] = pd.to_numeric(df_videos['like_count'], errors='coerce')
df_videos['view_count'] = pd.to_numeric(df_videos['view_count'], errors='coerce')
df_videos['comment_count'] = pd.to_numeric(df_videos['comment_count'], errors='coerce')


df_videos['likes_per_view'] = df_videos['like_count'] / df_videos['view_count']
df_videos['comments_per_view'] = df_videos['comment_count'] / df_videos['view_count']

# Pasamos a un df los logs de suscriptores
df_subscribers = pd.DataFrame(df_subscribers_list)



# Guardar los DataFrames en archivos CSV
df_videos.to_csv('youtube_videos.csv', index=False)
df_subscribers.to_csv('youtube_subscribers_history.csv', index=False, header=False)


In [57]:
len(df_videos)

25

In [42]:
from sqlalchemy import create_engine
import pandas as pd

# Credenciales de conexión a Redshift
DATABASE_TYPE = 'redshift+psycopg2'
DBAPI = 'psycopg2'
ENDPOINT = 'redshift-pda-cluster.cnuimntownzt.us-east-2.redshift.amazonaws.com'
USER = '2024_franco_santoliquido'
PASSWORD = 'L6^&9!2$xQ'
PORT = 5439
DATABASE = 'pda'

# Cadena de conexión a Redshift
engine = create_engine(f"{DATABASE_TYPE}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")

/var/folders/7_/ydy5jwh51735b2j1dqgskh7c0000gn/T/ipykernel_6267/2684085640.py:14: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'sqlalchemy_redshift.dialect.RedshiftDialect_psycopg2'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine(f"{DATABASE_TYPE}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")


In [44]:
df_videos.to_sql('2024_franco_santoliquido_schema.youtube_videos_stg', engine, index=False, if_exists='replace')
df_subscribers.to_sql('2024_franco_santoliquido_schema.youtube_subscribers_stg', engine, index=False, if_exists='replace') 

5

In [58]:
videos

[{'title': 'Chili Parker: Hay que autoinmolarse contra las ideas de la izquierda por el futuro de nuestros hijos',
  'published_at': '2024-09-22T16:27:42Z',
  'video_id': 'g1W6OEL2bW4',
  'video_type': 'none'},
 {'title': '🚨 CHILI PARKER EN LA MISA 🚨 LA MORCILLA CON BÁCULO y CHADMANZIO | La Misa de Dan',
  'published_at': '2024-09-21T02:26:15Z',
  'video_id': 'KGkJcg5FULQ',
  'video_type': 'none'},
 {'title': 'ZURDO QUEDA EN RIDÍCULO #lamisa #podcast #milei',
  'published_at': '2024-09-20T22:45:16Z',
  'video_id': 'KQf8hZj3P3U',
  'video_type': 'none'},
 {'title': 'ZURDOS UNIVERSITARIOS QUEDAN EN RIDÍCULO',
  'published_at': '2024-09-20T21:00:07Z',
  'video_id': 'cUHm5TYjfLg',
  'video_type': 'none'},
 {'title': '¿MEJOR PRESIDENTE? LO QUE NADIE TE CONTÓ DE JULIO.A.ROCA',
  'published_at': '2024-09-20T19:00:06Z',
  'video_id': 'bV-w9Z1SS4g',
  'video_type': 'none'}]

In [ ]:
import pandas as pd

# Obtenemos la información del canal
channel_info = get_channel_info(youtube, channel_id)

# Verificamos que se haya obtenido la información del canal
if channel_info:
    # Creamos una lista vacía para almacenar los datos combinados
    combined_data = []

    # Iteramos sobre los videos y sus estadísticas
    for video, stats in zip(videos, video_stats):
        # Combinamos la información de los videos, las estadísticas y el canal en un solo diccionario
        combined_data.append({
            'channel_name': channel_info['channel_name'],
            'channel_id': channel_info['channel_id'],
            'title': video['title'],
            'published_at': video['published_at'],
            'video_id': video['video_id'],
            'video_type': video['video_type'],
            'view_count': stats['view_count'],
            'like_count': stats['like_count'],
            'comment_count': stats['comment_count'],
            'duration_seconds': stats['duration'],
            'is_short': stats['is_short']
        })

    # Convertimos la lista de diccionarios en un DataFrame de pandas
    df_videos = pd.DataFrame(combined_data)


df['likes_per_view'] = df['like_count']/df['view_count']
df['comments_per_view'] = df['comment_count'] / df['view_count']



In [83]:
from googleapiclient.discovery import build
from datetime import datetime
from sqlalchemy import create_engine
import os
import time
from googleapiclient.errors import HttpError
import isodate


# Lista de canales a monitorear
CHANNEL_IDS = [
    'UC7mJ2EDXFomeDIRFu5FtEbA', 
    'UCvCTWHCbBC0b9UIeLeNs8ug',
    'UCWSfXECGo1qK_H7SXRaUSMg',
    'UCTHaNTsP7hsVgBxARZTuajw',
    'UC4mdhKZXjrKoq5aVG6juHEg' 
]


#Tres funciones para subir a redshift: una conecta, la otra uploadea en raw y la otra hace el upsert
def connect_to_redshift():

    DATABASE_TYPE = 'redshift+psycopg2'
    DBAPI = 'psycopg2'
    ENDPOINT = os.getenv('REDSHIFT_ENDPOINT')
    USER = os.getenv('REDSHIFT_USER')        
    PASSWORD = os.getenv('REDSHIFT_PASSWORD')
    PORT = 5439
    DATABASE = os.getenv('REDSHIFT_DATABASE')
    engine = create_engine(f"{DATABASE_TYPE}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
    return engine

def upload_to_redshift(engine, df, destintation_table, schema):
    df.to_sql(destintation_table, engine, schema, index=False, if_exists='replace')

def run_sql_queries(engine):
    # Obtener la ruta absoluta del archivo queries.sql
    base_dir = os.path.dirname(os.path.abspath(__file__))  # Obtiene la ruta absoluta de module_etl
    queries_file_path = os.path.join(base_dir, 'queries.sql')  # Se une para llegar a /queries.sql
    
    if not os.path.exists(queries_file_path):
        raise FileNotFoundError(f"No se encontró el archivo queries.sql en la ruta: {queries_file_path}")
    
    with open(queries_file_path, 'r') as file:
        sql_queries = file.read()
    queries = sql_queries.split(';')
    
    # Ejecuto las queries del archivo queries que tiene el upsert para hacer
    with engine.connect() as connection:
        for query in queries:
            query = query.strip()
            if query:  # Si la consulta no está vacía
                connection.execute(query)
                print(f"Ejecutada la consulta:\n{query}\n")


# Convierto duracion que da Youtube ISO 8601 a un objeto de tiempo
def convert_duration_to_seconds(duration_iso):

    duration = isodate.parse_duration(duration_iso)
    return int(duration.total_seconds())

# Inicio la api de Youtube con mis credenciales
def initialize_youtube_api():
    API_KEY = os.getenv('YOUTUBE_API_KEY')
    youtube = build('youtube', 'v3', developerKey=API_KEY)
    return youtube


# Funcion para buscar videos de un canal, con retrys

def get_videos_from_channel(youtube, channel_id, published_after, max_requests=10, sleep_time=1, max_retries=3):
    videos = []
    request_count = 0
    
    for attempt in range(max_retries):
        try:
            request = youtube.search().list(
                part='snippet',
                channelId=channel_id,
                publishedAfter=published_after,
                order='date',
                type='video'
            )
            response = request.execute()

            # De la respuesta sacamos titulo, momento de publicacion, id del video y tipo de video (live, upcoming o none)
            for item in response['items']:
                video_data = {
                    'title': item['snippet']['title'],
                    'published_at': item['snippet']['publishedAt'],
                    'video_id': item['id']['videoId'],
                    'video_type': item['snippet'].get('liveBroadcastContent', 'none')  # live, upcoming o none
                }
                videos.append(video_data)
            
            request_count += 1
            
            # Si se alcanza el límite de requests, metemos un sleep comentado
            if request_count % max_requests == 0:
                print(f"Realizadas {request_count} solicitudes. Esperando {sleep_time} segundos para continuar...")
                time.sleep(sleep_time)
            
            return videos

        except HttpError as e:
            print(f"Error al obtener videos del canal {channel_id}: {e}. Intento {attempt+1} de {max_retries}")
        except Exception as e:
            print(f"Ocurrió un error inesperado: {e}. Intento {attempt+1} de {max_retries}")
        
        # Meto otro sleep para retry
        time.sleep(sleep_time)
        
    raise Exception(f"No se pudo obtener los videos del canal {channel_id} después de {max_retries} intentos.")

#Busca la información del canal de Youtube que le pases
def get_channel_info(youtube, channel_id, max_retries=3, sleep_time=1):
    for attempt in range(max_retries):
        try:
            request = youtube.channels().list(
                part='snippet,statistics',
                id=channel_id
            )
            response = request.execute()

            # la respuesta tiene una key "items" contiene la data del canal y la cantidad de suscriptores
            channel_info = response['items'][0]
            channel_data = {
                'channel_name': channel_info['snippet']['title'],
                'channel_id': channel_info['id'],
                'subscriber_count': channel_info['statistics'].get('subscriberCount', 0)
            }

            return channel_data

        except HttpError as e:
            print(f"Error al obtener información del canal {channel_id}: {e}. Intento {attempt+1} de {max_retries}")
        except Exception as e:
            print(f"Ocurrió un error inesperado: {e}. Intento {attempt+1} de {max_retries}")
        
        # Esperamos antes de intentar nuevamente
        time.sleep(sleep_time)

    # Tiro el error para que pinche si no puedo obtener la información del canal
    raise Exception(f"No se pudo obtener la información del canal {channel_id} después de {max_retries} intentos.")

def get_video_statistics(youtube, video_ids, max_requests=10, sleep_time=2, max_retries=3):
    video_stats = []
    request_count = 0
    
    for attempt in range(max_retries):
        try:
            # Pasamos los IDs de los videos como una lista
            stats_request = youtube.videos().list(
                part='statistics,contentDetails',
                id=','.join(video_ids)
            )
            stats_response = stats_request.execute()

            # Procesamos las estadísticas
            for stats in stats_response['items']:
                duration_iso = stats['contentDetails'].get('duration', None)  # Ajuste aquí
                if duration_iso is None:
                    print(f"El video {stats['id']} no tiene duración, omitimos el dato.")
                    continue 

                duration_seconds = convert_duration_to_seconds(duration_iso)
                
                # Verificamos si el video es un YouTube Short (menos de 60 segundos)
                is_short = duration_seconds < 60
                
                stats_data = {
                    'video_id': stats['id'],
                    'view_count': stats['statistics'].get('viewCount', 0),
                    'like_count': stats['statistics'].get('likeCount', 0),
                    'comment_count': stats['statistics'].get('commentCount', 0),
                    'duration': duration_seconds,
                    'is_short': is_short
                }
                video_stats.append(stats_data)
            
            request_count += 1

            # Si se llega el límite de solicitudes, hacemos un sleep
            if request_count % max_requests == 0:
                print(f"Realizadas {request_count} solicitudes. Esperando {sleep_time} segundos para continuar...")
                time.sleep(sleep_time)

            return video_stats

        except HttpError as e:
            print(f"Error al obtener estadísticas de videos: {e}. Intento {attempt+1} de {max_retries}")
        except KeyError as e:
            print(f"Ocurrió un error de clave: {e}. Intento {attempt+1} de {max_retries}")
        except Exception as e:
            print(f"Ocurrió un error inesperado: {e}. Intento {attempt+1} de {max_retries}")
        
        # Esperamos para intentar nuevamente
        time.sleep(sleep_time)

    # Tiro el error para que pinche si no puedo obtener las estadísticas del canal
    raise Exception(f"No se pudieron obtener las estadísticas de videos después de {max_retries} intentos.")



# Función para sacar del json el datos que necesito
def transform_video_data(videos):
    transformed_data = []
    for video in videos:
        title = video['snippet']['title']
        published_at = video['snippet']['publishedAt']
        transformed_data.append({
            'title': title,
            'published_at': published_at
        })
    return transformed_data

# Función para agrupar datos por dia 
def group_videos_by_date(videos):
    grouped_data = {}
    for video in videos:
        date = video['published_at'].split("T")[0]
        if date not in grouped_data:
            grouped_data[date] = []
        grouped_data[date].append(video)
    return grouped_data


from datetime import datetime, timedelta
from sqlalchemy import create_engine
import pandas as pd

"""
    Tiene tres etapas:
    1. Iterar por cada channel y traer la información sobre qué videos se publicaron
    2. Iterar sobre los videos y traernos las estadísticas y las metemos en un DataFrame
    3. Subir los DataFrames (videos y suscriptores) a Redshift o guardarlos en CSV
    
    WARNING: Este código es para correr de manera diaria.
    Una vulnerabilidad que tiene es que la tabla con cantidad de suscriptores si se corre desde una fecha en particular, no va a obtener los subs desde esa fecha, si no de manera diaria
    
"""

# Inicializamos la API
youtube = initialize_youtube_api()

# Establezco la fecha de inicio (últimos N días)
published_after = (datetime.now() - timedelta(days=7)).isoformat("T") + "Z"

all_videos = []
all_video_stats = []
df_subscribers_list = []



# Se itera para cada channel de YouTube y sacamos estadísticas: cantindad de subs a la fecha de consulta, sus videos y para cada video métricas de views, comentarios y likes
for channel_id in CHANNEL_IDS:
    # Obtenemos la información del canal (nombre, id, suscriptores)
    channel_info = get_channel_info(youtube, channel_id)
    
    # Obtener la fecha de consulta actual
    consulta_fecha = datetime.now().strftime('%Y-%m-%d')
    
    # Crear un df con la info de suscriptores del canal
    df_subscribers_list.append({
        'channel_name': channel_info['channel_name'],
        'channel_id': channel_info['channel_id'],
        'consulta_fecha': consulta_fecha,
        'subscriber_count': channel_info['subscriber_count']
    })

    videos = get_videos_from_channel(youtube, channel_id, published_after)
    
    # Filtramos videos que contengan alguno de los campos que necesitamos para obtener el video ID. 
    # Hacemos esto por si cambia el nombre de la key con la que se guarda el video_id. 
    video_ids = []
    for video in videos:
        if 'video_id' in video:
            video_ids.append(video['video_id'])
        elif 'id' in video and isinstance(video['id'], dict) and 'videoId' in video['id']:
            video_ids.append(video['id']['videoId'])
        else:
            # Si no se encuentra el video_id, imprimimos el video para depuración
            print(f"Video sin 'video_id' o 'id' válido detectado: {video}")
    
    # Las stats de los videos
    video_stats = get_video_statistics(youtube, video_ids)
    
    # Agregamos los videos y las estadísticas a una lista combinada
    for video, stats in zip(videos, video_stats):
        video_id = video.get('video_id') or (video['id']['videoId'] if isinstance(video['id'], dict) and 'videoId' in video['id'] else 'N/A')
        
        all_videos.append({
            'channel_name': channel_info['channel_name'],
            'channel_id': channel_info['channel_id'],
            'title': video['title'],
            'published_at': video['published_at'],
            'video_id': video['video_id'],
            'video_type': video['video_type'],
            'view_count': stats['view_count'],
            'like_count': stats['like_count'],
            'comment_count': stats['comment_count'],
            'duration_seconds': stats['duration'],
            'is_short': stats['is_short']
        })

# Pasamos a un df los videos
df_videos = pd.DataFrame(all_videos)

# Hago una transformación de los datos 
df_videos['like_count'] = pd.to_numeric(df_videos['like_count'], errors='coerce')
df_videos['view_count'] = pd.to_numeric(df_videos['view_count'], errors='coerce')
df_videos['comment_count'] = pd.to_numeric(df_videos['comment_count'], errors='coerce')


df_videos['likes_per_view'] = df_videos['like_count'] / df_videos['view_count']
df_videos['comments_per_view'] = df_videos['comment_count'] / df_videos['view_count']

# Pasamos a un df los logs de suscriptores
df_subscribers = pd.DataFrame(df_subscribers_list)

#Iniciamos el engine
#engine_rs = connect_to_redshift()

#Inserto la raw data
#upload_to_redshift(engine_rs, df_videos, 'pda.2024_franco_santoliquido_schema.youtube_videos_stg')
#upload_to_redshift(engine_rs, df_subscribers, 'pda.2024_franco_santoliquido_schema.youtube_subscribers_stg')

#Corro lo que está en el file queries.sql que tiene el upsert a la tabla final



In [84]:
def connect_to_redshift():
    DATABASE_TYPE = 'redshift+psycopg2'
    DBAPI = 'psycopg2'
    ENDPOINT = 'redshift-pda-cluster.cnuimntownzt.us-east-2.redshift.amazonaws.com'
    USER = '2024_franco_santoliquido'
    PASSWORD = 'L6^&9!2$xQ'
    PORT = 5439
    DATABASE = 'pda'

    engine = create_engine(f"{DATABASE_TYPE}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
    return engine
def upload_to_redshift(engine, df, destintation_table, schema):
    df.to_sql(destintation_table, engine, schema, index=False, if_exists='replace')




In [85]:

# Ejecutar las queries directamente usando SQLAlchemy
queries = """
-- Update e insert para la tabla de videos
UPDATE pda."2024_franco_santoliquido_schema".BT_YOUTUBE_VIDEO_STATS DW
SET 
    YT_VIEW_COUNT_7D = STG.view_count,
    YT_LIKE_COUNT_7D = STG.like_count,
    YT_COMMENT_COUNT_7D = STG.comment_count,
    AUD_UPD_ID = CURRENT_DATE,
    AUD_UPDATED_FROM = 'ETL_YOUTUBE_CDE_PDA'
FROM 
    pda."2024_franco_santoliquido_schema".youtube_videos_stg STG
WHERE  
    STG.video_id = DW.YT_VIDEO_ID
    AND STG.published_at >= DATEADD(day, -7, CURRENT_DATE)
    ;

INSERT INTO pda."2024_franco_santoliquido_schema".BT_YOUTUBE_VIDEO_STATS (
    YT_CHANNEL_NAME,
    YT_CHANNEL_ID,
    YT_TITLE_NAME,
    YT_DATE_PUBLISHED,
    YT_VIDEO_ID,
    YT_VIDEO_TYPE,
    YT_VIEW_COUNT_7D,
    YT_LIKE_COUNT_7D,
    YT_COMMENT_COUNT_7D,
    YT_DURATION_SECS,
    YT_IS_SHORT_FLAG,
    AUD_UPD_ID,
    AUD_INS_DATE,
    AUD_UPDATED_FROM
)
SELECT 
    STG.channel_name,
    STG.channel_id,
    STG.title,
    STG.published_at,
    STG.video_id,
    STG.video_type,
    STG.view_count,
    STG.like_count,
    STG.comment_count,
    STG.duration_seconds,
    STG.is_short,
    CURRENT_DATE,
    CURRENT_DATE, 
    'ETL_YOUTUBE_CDE_PDA'
FROM 
    pda."2024_franco_santoliquido_schema".youtube_videos_stg STG
LEFT JOIN 
    pda."2024_franco_santoliquido_schema".BT_YOUTUBE_VIDEO_STATS DW
    ON STG.video_id = DW.YT_VIDEO_ID
WHERE 
    DW.YT_VIDEO_ID IS NULL;

-- Insert para la tabla de suscriptores

INSERT INTO pda."2024_franco_santoliquido_schema".LG_CHANNEL_SUBSCRIBERS (
    YT_CHANNEL_NAME,
    YT_CHANNEL_ID,
    YT_DATE_ID,
    YT_SUSCRIBER_COUNT,
    AUD_INS_DATE,
    AUD_UPDATED_FROM
)
SELECT 
    STG.channel_name,
    STG.channel_id,
    STG.consulta_fecha,
    STG.subscriber_count,
    CURRENT_DATE,
    'ETL_YOUTUBE_CDE_PDA'
FROM 
    pda."2024_franco_santoliquido_schema".youtube_subscribers_stg STG
LEFT JOIN 
    pda."2024_franco_santoliquido_schema".LG_CHANNEL_SUBSCRIBERS DW
ON 
    STG.channel_id = DW.YT_CHANNEL_ID
    AND STG.consulta_fecha = DW.YT_DATE_ID
WHERE 
    DW.YT_CHANNEL_ID IS NULL
    ;
"""


In [86]:
# Suponiendo que ya tienes tus DataFrames creados (df_videos y df_subscribers):
# Sube los DataFrames con el nombre del esquema ajustado

upload_to_redshift(engine, df_videos, 'youtube_videos_stg', schema='2024_franco_santoliquido_schema' )
upload_to_redshift(engine, df_videos, 'youtube_subscribers_stg', schema='2024_franco_santoliquido_schema' )


In [76]:
df_videos.head()

,channel_name,channel_id,title,published_at,video_id,video_type,view_count,like_count,comment_count,duration_seconds,is_short,likes_per_view,comments_per_view
0,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,BETU ES EL NUEVO REY DE LOS MACARONS,2024-10-05T22:00:33Z,gtTM9GJ8nZU,none,114,39,0,58,True,0.342105,0.000000
1,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,EL PELAO FILOSOFÓ SOBRE EL MATE,2024-10-05T20:00:08Z,OppCmaXtZME,none,197,54,0,54,True,0.274112,0.000000
2,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,HOMI ELEVANDO EL NIVEL DEL PROGRAMA,2024-10-05T18:00:11Z,Q3LCtQoVFfE,none,3214,327,2,52,True,0.101742,0.000622
3,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,EL MITO DEL PETISO PIJUDO EN HABLEMOS SIN SABER,2024-10-05T16:00:16Z,J8KDqUj9B2g,none,7745,552,3,58,True,0.071272,0.000387
4,OLGA,UC7mJ2EDXFomeDIRFu5FtEbA,NATI CORRIÓ DROGADA POR ÁMSTERDAM,2024-10-05T12:00:28Z,FJb5YHpWoDI,none,8722,507,8,47,True,0.058129,0.000917
